In [1]:
import pandas as pd
import nltk
nltk.download('punkt')
from transformers import pipeline
from rouge_score import rouge_scorer

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/jusepulveda/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# Function to generate a summary using BART
def extractive_summary(question, answer):
    # Instead of scoring sentences, use BART to summarize the entire answer
    result = summarization_pipeline(answer, max_length=150, min_length=50, do_sample=False)
    summary = result[0]['summary_text']  # Extract the summarized text
    return summary

# Function to calculate ROUGE scores for evaluating the summary
def evaluate_summary(reference, generated_summary):
    scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)
    scores = scorer.score(reference, generated_summary)
    return scores

In [ ]:
path = '../data/medquad.csv'
df=pd.read_csv(path)
print(df.head(2))

In [ ]:
# Initialize BART-based summarization pipeline
summarization_pipeline = pipeline("summarization", model="facebook/bart-large-cnn")

In [ ]:
index = 0  # Change this to the row number you want
question = df.loc[index, 'question']
answer = df.loc[index, 'answer']

# Apply extractive summarization to the single entry
summary = extractive_summary(question, answer)

# Display the results
print(f"Question: {question}")
print(f"Original Answer: {answer}")
print(f"Summary: {summary}")

# Evaluate the summary using ROUGE scores
rouge_scores = evaluate_summary(answer, summary)  # Comparing to the original answer
print(f"ROUGE Scores: {rouge_scores}")